In [1]:
# utils 
import torch

# data 
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

# model
import torch.nn as nn
import torch.nn.functional as F

# training
import torch.optim as optim
import tqdm

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

### Data Preparation

In [4]:
# creating text and label fields
TEXT = Field(lower=True, tokenize="spacy", batch_first=True)
LABEL = LabelField(batch_first=True, dtype=torch.float)

In [5]:
# download the dataset
train, val = datasets.IMDB.splits(text_field=TEXT, label_field=LABEL)

In [6]:
# build the vocabulary
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [7]:
# data loaders
BATCH_SIZE = 128
train_loader, val_loader = BucketIterator.splits(
    datasets=(train, val),
    batch_sizes=(BATCH_SIZE, BATCH_SIZE),
    device=device
    
)

In [8]:
for batch in train_loader:
  print(type(batch))
  break

<class 'torchtext.data.batch.Batch'>


### Model Building

In [12]:
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super(Model, self).__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [62]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [63]:
model = Model(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX).to(device)

In [64]:
out = model(batch.text)

In [65]:
print(out.shape)

torch.Size([128, 1])


### Training

In [66]:
def accuracy(y, y_):
  correct = 0
  for i in range(y.shape[0]):
    if y[i]==y_[i]:
      correct += 1
  return correct/y.shape[0]

In [67]:
def eval(model, data, criterion):
  losses = []
  acc = []
  
  with torch.no_grad():
    for batch in data:
      text, label = batch.text, batch.label
      outputs = model(text)
      l = criterion(outputs.view(-1), label.view(-1))
      a = accuracy(label, outputs.view(-1) > 0.5)
      losses.append(l.item())
      acc.append(a)
    
  return sum(losses)/len(losses), sum(acc)/len(acc)


In [68]:
lr = 1e-3
EPOCHS = 10

In [69]:
# optimizer and criterion
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(params=model.parameters(), lr=lr)

In [70]:
epoch_progress = tqdm.tqdm(total=EPOCHS, desc="Epoch", position=0)
total_steps = len(train_loader)
steps = 0
for epoch in range(EPOCHS):

  epoch_loss = []
  epoch_acc = []
  step_progress = tqdm.tqdm(total=len(train_loader), desc="Epoch", position=0)
  for batch in train_loader:
    text, label = batch.text, batch.label
    outputs = model(text)

    optimizer.zero_grad()
    loss = criterion(outputs.view(-1), label.view(-1))
    loss.backward()
    optimizer.step()

    acc = accuracy(label, outputs.view(-1) > 0.5)

    if steps%100==0:
      print(f'Step {steps}/{len(train_loader)*EPOCHS} | Train_loss {loss.item():.4f} | Train_acc {acc:.4f}')


    epoch_loss.append(loss.item())
    epoch_acc.append(acc)
    steps +=1 
    step_progress.update(1)



  avg_loss = sum(epoch_loss)/len(epoch_loss)
  avg_acc = sum(epoch_acc)/len(epoch_acc)

  v_loss, v_acc = eval(model, val_loader, criterion)

  print(f'Epoch {epoch}/{EPOCHS} | Train_loss {avg_loss:.4f} | Train_acc {avg_acc:.4f} | Val_loss {v_loss:.4f} | Val_acc {v_acc:.4f}')
  epoch_progress.update(1)


Epoch:   1%|          | 2/196 [00:00<00:34,  5.66it/s]

Step 0/1960 | Train_loss 1.1310 | Train_acc 0.5078


Epoch:  52%|█████▏    | 102/196 [00:11<00:09,  9.67it/s]

Step 100/1960 | Train_loss 0.3305 | Train_acc 0.5703


Epoch:   1%|          | 1/196 [00:00<00:26,  7.41it/s]

...

Step 1800/1960 | Train_loss 0.1200 | Train_acc 0.8750


Epoch:  70%|███████   | 138/196 [00:15<00:06,  9.25it/s]

Step 1900/1960 | Train_loss 0.0963 | Train_acc 0.8906


Epoch: 100%|██████████| 10/10 [09:03<00:00, 54.37s/it]

Epoch 9/10 | Train_loss 0.0994 | Train_acc 0.8972 | Val_loss 0.1369 | Val_acc 0.8222


In [71]:
eval(model, val_loader, criterion)

(0.13683705818744338, 0.823126594387755)